In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.dpi':180})

import sys
sys.path.append("/home/msdos/DESI-QA/desiarc-main/arc")
sys.path.append("/home/msdos/DESI-QA/")
sys.path.append("/home/msdos/DESI-QA/output/figures/")
import find_center as fc
from spotfinder import spotfinder
from xylib import calc_movetables as cm
import reportlib as rl

figPath = '/home/msdos/DESI-QA/output/figures/Backlash test/'

In [3]:
thetaTime = '2023-07-06 16:09:22'
phiTime = '2023-07-07 11:20:35'
testStart = np.datetime64(phiTime)
testFinish = testStart + np.timedelta64(1,"h")

# Setting paths
datapath = "../output/database.csv"
fidpath = "../output/fiddb.csv"
df = rl.importToDf(datapath,fidpath,testStart,testFinish,testType='RS',motor='phi')
ramp=1.995
firstBacklash = 10+ramp-rl.angle_between(tuple(df[['xc2mm','yc2mm']].loc[0]),tuple(df.loc[0][['x_mm','y_mm']]),tuple(df.loc[1][['x_mm','y_mm']]))

Adding session labels for testType=RS
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [4]:
df['Backlash'].loc[1] = firstBacklash

/data/common/software/products/python-3.6.15.Linux64/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
df[['Backlash','angle']]

,Backlash,angle
0,NaN,0.0
1,-1.920047,10.0
2,-1.452246,10.0
3,-1.482709,10.0
4,-1.466624,10.0
5,-1.706508,45.0
6,-1.611482,45.0
7,-1.629237,45.0
8,-1.608351,45.0
9,-1.577957,90.0


In [6]:
np.nanmedian(df['Backlash'])

-1.6040542986972213

# For theta

In [7]:
testStart = np.datetime64(thetaTime)
testFinish = testStart + np.timedelta64(1,"h")

df = rl.importToDfBacklash(datapath,fidpath,testStart,testFinish,testType='RS',motor='theta')

Adding session labels for testType=RS


In [25]:
smalldf = df[['x_mm',"y_mm","xc2mm","yc2mm","angle"]]
ramp = 1.995

In [24]:
smalldf

,x_mm,y_mm,xc2mm,yc2mm,angle
0,63.876037,32.232067,69.848263,31.512703,0.0
1,63.880115,30.764506,69.848263,31.512703,10.0
2,63.861633,32.046881,69.848263,31.512703,10.0
3,63.876836,30.762762,69.848263,31.512703,10.0
4,63.859765,32.045919,69.848263,31.512703,10.0
5,69.536489,25.504871,69.848263,31.512703,90.0
6,63.852805,32.043149,69.848263,31.512703,90.0
7,69.527477,25.502456,69.848263,31.512703,90.0
8,63.847827,32.041373,69.848263,31.512703,90.0
9,75.856144,31.196726,69.848263,31.512703,180.0


In [52]:
backlash_arr = np.array([],dtype=float)
center = tuple(smalldf.loc[0][['xc2mm','yc2mm']])
for k in range(1,len(df)-1):
    current = tuple(smalldf.loc[k+1][['x_mm','y_mm']])
    past = tuple(smalldf.loc[k][['x_mm','y_mm']])
    angle = rl.angle_between(center,current,past)
    requested = smalldf.loc[k+1]['angle']+2*ramp
    if requested>180:
        angle = 360-angle
    backlash = requested - angle
#     print(k+1,f"{angle:.5f}",f"{requested:.3f}")
#     print(k+1,backlash)
    backlash_arr = np.append(backlash_arr,backlash)

In [53]:
np.median(backlash_arr)

1.814610203121049

# Redundant

In [18]:
center = (0,0)
point1 = (1,0)
point2 = (-0.5,0)

rl.angle_between_extended(center,point1,point2)

180.0

In [ ]:
df[df['angle']==180][['x_mm','y_mm','direction']]

In [ ]:
def costheta(center,point1,point2):
    a = np.array(center) - np.array(point1)
    a = np.dot(a,a)/np.sqrt(np.dot(a,a))
    b = np.array(point2) - np.array(center)
    b = np.dot(b,b)/np.sqrt(np.dot(b,b))
    c = np.array(point2) - np.array(point1)
    c = np.dot(c,c)/np.sqrt(np.dot(c,c))
    return np.degrees(np.arccos((c**2 - a**2 - b**2)/(-2 * a * b)))

In [ ]:
RS_Backlash = np.array([np.nan],dtype=float)

ramp = 1.995

for i in range(0,len(df)-1):
    degree = costheta((df.loc[i]['xc2mm'],df.loc[i]['yc2mm']),(df.loc[i]['x_mm'],df.loc[i]['y_mm']),
             (df.loc[i+1]['x_mm'],df.loc[i+1]['y_mm']))
    if df.loc[i+1]['angle']>=180: # axes flip for 180 deg and greater moves
        degree = 360-degree
    if i==0:
        degree-=ramp
    print(degree,df.loc[i+1]['angle'])
    RS_Backlash = np.append(RS_Backlash,degree-df.loc[i+1]['angle'])
    
df["Backlash"] = RS_Backlash

In [ ]:
np.nanmedian(df['Backlash'])

In [ ]:
RS_Backlash

In [ ]:
plt.hist(RS_Backlash)
plt.show()

In [ ]:
plt.figure(figsize=[8,6])

save = True

title="Backlash for phi - away from home, RS Procedure"

plt.grid(linestyle='--')
plt.suptitle(title+"\n Median value: "+f"{np.nanmedian(df['Backlash']):.3f} degrees")
plt.ylabel("Median Backlash [deg]")
plt.xlabel("Commanded angle [deg]")
for angles in np.unique(df['angle'])[1:]:
    mask = df['angle']==angles
    scatkwargs1 = dict(marker='o',s=10)
#     scatkwargs2 = dict(yerr=np.std(df[msk]['Backlash']),marker='o',markersize=5, elinewidth=0, capsize=4)
#     msk1 = (df['direction']=='ccw') & (df['angle']==angle)
#     msk2 = (df['direction']=='cw') & (df['angle']==angle)
    plt.scatter(df[mask]['angle'], df[mask]['Backlash'],**scatkwargs1,color='red')
#     plt.errorbar(angle, np.median(df[msk1]['Backlash']),ecolor='blue',**scatkwargs2,color='blue',alpha=0.5)
#     plt.errorbar(angle, np.median(df[msk2]['Backlash']),ecolor='green',**scatkwargs2,color='green',alpha=0.5)

    plt.errorbar(angles,df[mask]['Backlash'].mean(),yerr=df[df['angle']==angles]['Backlash'].std(),c='black',
                 marker='x',markersize=5, elinewidth=0, capsize=4,zorder=0)
    
# plt.xticks(np.arange(0,361,step=40))
# plt.xlim(0,360)
# plt.ylim(-2.2,-1.7)

plt.tight_layout()

if save:
    plt.savefig(figPath+title+".jpg",dpi=180)

plt.show()